# NFL Project
This is a (currently incomplete) hobby project for messing around with NFL statistics.

My original intent for this project was to find a way to separate the performance of a wide receiver (WR) from his quarterback (QB) in order to settle frequent debates with firends.  Since the two positions are so closely linked, it can be difficult to determine how good a WR is relative to his peers if he plays with an elite QB, and vice versa.  In order to answer this question, I create a metric to evaluate a WR's and QB's performance relative to the rest of the NFL.

## Configuration
The code reads in settings from a configuration file located in `config/config.cfg`:
1. Paths: specify the paths to the home directory of the project and the directory where the data is located
2. Config: a list of players to be plotted, specified via their ID that can be looked up in `data/playerIDs.cfg`
3. Metric: here the weights of various stats can be set for calculating the metric

## Running the code
The code is exucuted by running the `calculate_nfl_averages.py` script in the root directory, and it reads inputs from `config/config.cfg`

Below is the console output from running the code:
```
$ python3 calculate_nfl_averages.py 
Processing play #25000
Processing play #50000
Processing play #75000
Processing play #100000
Processing play #125000
Processing play #150000
Processing play #175000
Processing play #200000
Processing play #225000
Processing play #250000
Processing play #275000
Processing play #300000
Processing play #325000
Finished reading 342276 plays in 9.52 seconds.
Calculating stats for player #1000
Calculating stats for player #2000
Finished calculating stats for 2417 players in 4.48 seconds.
Finished calculating league averages in 0.13 seconds.
Finished calculating metric for all players in 0.61 sec.
Making plots for player 2506106 -- Larry Fitzgerald (WR).
Done in 13.36 sec.
Making plots for player 2503532 -- Hines Ward (WR).
Done in 12.40 sec.
Making plots for player 2505587 -- Anquan Boldin (WR).
Done in 12.72 sec.
```